In [2]:
import torch.nn as nn
import torch
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append('../mnist')
import matplotlib.pyplot as plt

import cnn_model as model
from torch.autograd import Variable


In building a model you will need: 
- a model architecture with model parameters
- an optimizer with optimizer parameters
- a training and a testing dataset


The training and testing dataset is fed to the model in batches (of size specified below) using data loaders.

In [3]:
batch_size_train = 256
batch_size_test = 256

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True,pin_memory=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True,pin_memory=True)

The model and the model training are specified by a parameters dictionary. Very little validation or sanity checking is done on the parameters at this point (#TODO). If no parameters are specified, a set of default ones will be used.

In [7]:
parameters = {
            'max_pool_kernel_size':2,
            'max_pool_stride':2,
            'max_pool_padding':0,

            'conv1_kernel_size':3,
            'conv1_stride':1,
            'conv1_padding':1,
            'conv1_in_channels':1,
            'conv1_out_channels':32,

            'conv2_kernel_size':3,
            'conv2_stride':1,
            'conv2_padding':1,
            'conv2_in_channels':32,
            'conv2_out_channels':64,

            'conv3_kernel_size':3,
            'conv3_stride':1,
            'conv3_padding':1,
            'conv3_in_channels':64,
            'conv3_out_channels': 256,

            'linear1_output':1000,
            'linear2_output':500,

            'learning_rate':0.005,
            'momentum':0.5
}

The model architecture and the optimizers can be defined and build with the provided constructors. Training and evaluating is done by calling the relevant methods.

In [ ]:
net = model.Model(parameters)

optimizer = model.default_optimizer(net, learning_rate=0.005, momentum=0.5)
num_epochs = 1
use_cuda = False
if use_cuda:
    net = net.cuda()
if use_cuda and torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)
model.train_model(net, num_epochs,optimizer, train_loader, log_interval = 50,use_cuda=use_cuda)
model.eval_model(net, test_loader, use_cuda=use_cuda)

Train epoch: 1 [0/60000 (0%)]	 Loss: 2.303548


Optimal (where optimality depends on the chosen metric) parameters can be found by hyperparameter tuning. This is done by sequentially (not parallely at this point) evaluating a set of parameters. In the example below, we are optimizing on the number of channels of the third convolutional layer.

In [4]:
parameters_to_tune = {
            'max_pool_kernel_size':2,
            'max_pool_stride':2,
            'max_pool_padding':0,

            'conv1_kernel_size':3,
            'conv1_stride':1,
            'conv1_padding':1,
            'conv1_in_channels':1,
            'conv1_out_channels':32,

            'conv2_kernel_size':3,
            'conv2_stride':1,
            'conv2_padding':1,
            'conv2_in_channels':32,
            'conv2_out_channels':64,

            'conv3_kernel_size':3,
            'conv3_stride':1,
            'conv3_padding':1,
            'conv3_in_channels':64,
            'conv3_out_channels': [32,64,128,256],

            'linear1_output':1000,
            'linear2_output':500,

            'learning_rate':0.005,
            'momentum':0.5
}
optimal = model.hyperpameter_tuning(test_loader, train_loader,parameters_to_tune,'accuracy')

{'max_pool_kernel_size': 2, 'max_pool_stride': 2, 'max_pool_padding': 0, 'conv1_kernel_size': 3, 'conv1_stride': 1, 'conv1_padding': 1, 'conv1_in_channels': 1, 'conv1_out_channels': 32, 'conv2_kernel_size': 3, 'conv2_stride': 1, 'conv2_padding': 1, 'conv2_in_channels': 32, 'conv2_out_channels': 64, 'conv3_kernel_size': 3, 'conv3_stride': 1, 'conv3_padding': 1, 'conv3_in_channels': 64, 'conv3_out_channels': [32, 64, 128, 256], 'linear1_output': 1000, 'linear2_output': 500, 'learning_rate': 0.005, 'momentum': 0.5}
current parameters:  {'max_pool_kernel_size': 2, 'max_pool_stride': 2, 'max_pool_padding': 0, 'conv1_kernel_size': 3, 'conv1_stride': 1, 'conv1_padding': 1, 'conv1_in_channels': 1, 'conv1_out_channels': 32, 'conv2_kernel_size': 3, 'conv2_stride': 1, 'conv2_padding': 1, 'conv2_in_channels': 32, 'conv2_out_channels': 64, 'conv3_kernel_size': 3, 'conv3_stride': 1, 'conv3_padding': 1, 'conv3_in_channels': 64, 'conv3_out_channels': 32, 'linear1_output': 1000, 'linear2_output': 500, 

/home/remo_storni/projects/pytorch_mnist/cnn_model.py:126: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  so = F.log_softmax(o)
/opt/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3016, Accuracy: 1160/10000 (11%)

Train epoch: 1 [0/60000 (0%)]	 Loss: 2.303574
Train epoch: 1 [2560/60000 (4%)]	 Loss: 2.296616
Train epoch: 1 [5120/60000 (9%)]	 Loss: 2.297343
Train epoch: 1 [7680/60000 (13%)]	 Loss: 2.293498
Train epoch: 1 [10240/60000 (17%)]	 Loss: 2.292812
Train epoch: 1 [12800/60000 (21%)]	 Loss: 2.290650
Train epoch: 1 [15360/60000 (26%)]	 Loss: 2.288625
Train epoch: 1 [17920/60000 (30%)]	 Loss: 2.283330
Train epoch: 1 [20480/60000 (34%)]	 Loss: 2.279321
Train epoch: 1 [23040/60000 (38%)]	 Loss: 2.273386
Train epoch: 1 [25600/60000 (43%)]	 Loss: 2.275061
Train epoch: 1 [28160/60000 (47%)]	 Loss: 2.267590
Train epoch: 1 [30720/60000 (51%)]	 Loss: 2.261456
Train epoch: 1 [33280/60000 (55%)]	 Loss: 2.254593
Train epoch: 1 [35840/60000 (60%)]	 Loss: 2.246174
Train epoch: 1 [38400/60000 (64%)]	 Loss: 2.241992
Train epoch: 1 [40960/60000 (68%)]	 Loss: 2.231632
Train epoch: 1 [43520/60000 (72%)]	 Loss: 2.226314
Train epoch: 1 [46080/60000 (77%)]

In [5]:
print(optimal)

(0.5063, {'max_pool_kernel_size': 2, 'max_pool_stride': 2, 'max_pool_padding': 0, 'conv1_kernel_size': 3, 'conv1_stride': 1, 'conv1_padding': 1, 'conv1_in_channels': 1, 'conv1_out_channels': 32, 'conv2_kernel_size': 3, 'conv2_stride': 1, 'conv2_padding': 1, 'conv2_in_channels': 32, 'conv2_out_channels': 64, 'conv3_kernel_size': 3, 'conv3_stride': 1, 'conv3_padding': 1, 'conv3_in_channels': 64, 'conv3_out_channels': 64, 'linear1_output': 1000, 'linear2_output': 500, 'learning_rate': 0.005, 'momentum': 0.5})
